In [2]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError,ChunkedEncodingError
import itertools
import multiprocessing
from IPpool import GetHeaders
from IPpool import Ip_pool
import random
import copy
import os
# import requests_toolbelt.adapters.appengine

# requests_toolbelt.adapters.appengine.monkeypatch()


importing Jupyter notebook from IPpool.ipynb


In [3]:
def requests_visit_url_test(url,is_use_ip =False,timeout=5):
        global ip_pool
        global bad_ip
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                    'Accept-Encoding':'gzip, deflate',
                    'Accept-Language':'zh-CN,zh;q=0.8,en;q=0.6',
                    'Cache-Control':'max-age=0',
                    'Connection':'keep-alive',
                    'Cookie':'_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; __mta=222713764.1505836341840.1507819135553.1507819166825.4; wed_user_path=163|0; Hm_lvt_dbeeb675516927da776beeb1d9802bd4=1508317369; __utma=1.1801105885.1505634423.1505636820.1508318009.3; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; JSESSIONID=E0D0810114F31617ED97BC416CFDCE80; aburl=1; cy=1; cye=shanghai; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; s_ViewType=10; _lxsdk_s=15f304e1839-37c-2e4-6cf%7C%7C174',                    'Host':'www.dianping.com',
                    'If-Modified-Since':'Wed, 18 Oct 2017 08:58:29 GMT',
                    'Referer':'http://www.dianping.com/search/category/219/30/g141',
                    'Upgrade-Insecure-Requests':'1',
                    'User-Agent':random.choice(GetHeaders().user_agent_list)}

        #获取任意一个请求头
        #headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        bad_ip = []  #用来装坏的ip
        
        #*********如果不设置ip的话**********************************
        if is_use_ip == False:
            res = requests.get(url,headers = headers,timeout=timeout)  #访问url，不设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            if soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                print("ip被禁止了！！！！！！！！！！！！！！！！")
                raise ConnectionError
           
        #************如果设置ip的话，则执行这一段*********************
        else:
            is_continue = True   #设置循环标志
            while is_continue:
            #整理成ip地址的格式
                try:
                    #
                    print(len(bad_ip))
                    if len(ip_pool)==len(bad_ip):
                        ip_url = ['http://www.xicidaili.com/nn/']
                        test_url = 'http://www.dianping.com/'
                        ip_pool= Ip_pool().get_ip_pool(test_url,timeout=2)
                        prox_ip = random.choice(ip_pool)
                        bad_ip = []
                        
                    #******随机生成一个ip*******
                    prox_ip = random.choice(ip_pool)
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    
                    #*******用代理ip访问********
                    res = requests.get(url,headers = headers,proxies = proxies,timeout=timeout) #设代理ip访问
                    html =  res.content.decode('utf-8')
                    soup  =  BeautifulSoup(html,'lxml')
                    
                    
                    #********如果返回网页被禁止的情况，触发异常,否则返回正常**********
                    if  soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                        print("ip被禁止了!!!!!!!!!!!!!!!!!!!!")
                        raise ConnectionError
                    else:
                        is_continue = False

                except HTTPError as e:
                    bad_ip.append(prox_ip)
                    print("断网")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)


                except Timeout as e:
                    bad_ip.append(prox_ip)
                    print("超时")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)


                except ConnectionError as e:
                    bad_ip.append(prox_ip)
                    print("访问被拒")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)
                    
                except AttributeError as e:
                    bad_ip.append(prox_ip)
                    print("----------")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)

        return soup

In [4]:
#**************将装机地址变成一组一组*****************
def get_chunk(adr_ls,n):
    new_adr_ls = copy.deepcopy(adr_ls)
    adr_chunk = [new_adr_ls[i:i + n] for i in range(0, len(new_adr_ls), n)]
    return adr_chunk

In [5]:
#缓存结果
def save_cache(result,filename):
    with open(filename,'wb') as f1:
        pickle.dump(result,f1)

In [6]:
#加载缓存
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

In [7]:
# 切换路径
def get_spc_path(father_path,child_path):
    os.chdir(father_path)
    if os.path.exists(child_path):
        os.chdir(child_path)
    else:
        os.mkdir(child_path)
        os.chdir(child_path)
    print("done!!")